Links to read:
- [Word embeddings Wiki](https://en.wikipedia.org/wiki/Word_embedding)
- [Gensim Python library](https://en.wikipedia.org/wiki/Gensim)
- [Long-Short term memory (LSTM)](https://www.datacamp.com/community/tutorials/lstm-python-stock-market#lstm)